## 규칙
- 각 이름은 띄어쓰기 없이 작성하기. 예를 들어, "영업 시간"이 아니라 "영업시간"으로

In [2]:
from tqdm import tqdm
from langchain_community.vectorstores import FAISS
import pickle
import pandas as pd
import numpy as np
from langchain.schema import Document
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.document_loaders import UnstructuredExcelLoader
from langchain_community.document_loaders import DataFrameLoader
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings

In [3]:
class CFG:
    # store="프랭크버거"
    output_path = "/home/user09/beaver/data/db"
    data_path = ""
    save_path = ""
    embedding_model= "nlpai-lab/KoE5" # "BAAI/bge-m3"
    retriever_k=5
    retriever_bert_weight=0.7
    version='4_general'
    # info_type='STORE_INFO'   # STORE_INFO, TIME_INFO, MENU_INFO
    store_num = 102496  # 102496, 102506, 103807, 104570, 104933
    fill_nan = "None"  # 없는 정보의 경우에는 "정보없음"보다는 "None"이 나은 듯
    # basic_info = ["주차장", "씨씨티비", "영업시간", "예약가능여부", "전화번호"]
    seed=42


CFG.data_path = f"/home/user09/beaver/data/shared_files/dataset/dataset_v{CFG.version}.xlsx"
print(CFG.data_path)
CFG.save_path = f"/home/user09/beaver/data/shared_files/dataset/dataset_v{CFG.version}_{CFG.store_num}_preprocessed.xlsx"
print(CFG.save_path)

/home/user09/beaver/data/shared_files/dataset/dataset_v4_general.xlsx
/home/user09/beaver/data/shared_files/dataset/dataset_v4_general_102496_preprocessed.xlsx


### STORE_INFO

In [4]:
################ 전처리 ################
from langchain.schema import Document

# columns_to_use = ['STORE_NO', 'STORE_NM', 'ADDR', 'DETAIL_ADDR', 'TEL', 'GUID_CONTS', 'PAYMNT_MN_CD']
columns_to_use = ['STORE_NO', 'STORE_NM', 'ADDR', 'DETAIL_ADDR', 'TEL', 'PAYMNT_MN_CD']  # 'GUID_CONTS'
df_store = pd.read_excel(CFG.data_path, sheet_name='STORE_INFO', usecols=columns_to_use, dtype={"TEL": str})

df_store = df_store[df_store['STORE_NO']==CFG.store_num]  # 해당 매장 정보만 가져옴
store = df_store['STORE_NM'].unique()[0]
print(store)
df_store.drop(columns=['STORE_NO', 'STORE_NM'], inplace=True)

# for col in df_store.columns:   # 매장 정보는 매장 당 한 개의 row만 존재하므로 제공해주지 않는 매장 정보 컬럼은 삭제
#     if df_store[col].isnull().values.any():   
#         df_store.drop(columns=[col], inplace=True)

# df_2.drop(columns=['STORE_NO'], inplace=True)
# print(df_2)

# 1. 주소와 상세 주소 합치기
df_store['ADDR'] = df_store['ADDR'] + ' ' + df_store['DETAIL_ADDR']
df_store['ADDR'] = df_store['ADDR'].fillna(CFG.fill_nan)   # !!!!!!!!!!!!!!!
df_store.drop(columns=['DETAIL_ADDR'], inplace=True)

df_store.fillna(CFG.fill_nan, inplace=True)  # !!!!!!!!!!!!!!!

# 2. 결제수단 매핑
def map_payment_methods(payment_codes):
    payment_mapping = {
    'CA': '현금',
    'CD': '신용카드',
    'PO': '포인트',
    'PC': '간편결제',
    'GV': '모바일상품권',
    'ET': '기타결제'
    }
    codes = payment_codes.split(', ')
    mapped_names = [payment_mapping[code] for code in codes if code in payment_mapping]
    return ', '.join(mapped_names)

df_store['PAYMNT_MN_CD'] = df_store['PAYMNT_MN_CD'].apply(map_payment_methods)
df_store_col = df_store.columns

# 결과 출력
print(df_store)
print(df_store_col)

히스피커피 서면본점
                     ADDR          TEL   PAYMNT_MN_CD
0  부산 부산진구 중앙대로 679-8 히스피  01098007047  현금, 신용카드, 포인트
Index(['ADDR', 'TEL', 'PAYMNT_MN_CD'], dtype='object')


In [5]:
############## 컬럼명 변경 ##############
store_column_mapping = {
    'STORE_NO': '상점번호',
    'STORE_NM': '상점명',
    'ADDR': '위치',
    'DETAIL_ADDR': '매장주소',
    'TEL': '전화번호',
    'X': '좌표 X',
    'Y': '좌표 Y',
    'GUID_TTL': '안내제목',  # !!!!!!!!!!!!!!!!!
    'GUID_CONTS': '안내내용',
    'PAYMNT_MN_CD': '결제수단',
}

for col in df_store_col:
    df_store = df_store.rename(columns={col: store_column_mapping[col]})
    
print(df_store)

                       위치         전화번호           결제수단
0  부산 부산진구 중앙대로 679-8 히스피  01098007047  현금, 신용카드, 포인트


In [6]:
############### 한 컬럼으로 통합 ###############
def create_contents_rowwise(df):
    contents_data = []
    for _, row in df.iterrows():
        for col in df.columns:
            contents_data.append(f"'{col}': {row[col]}")
    new_df = pd.DataFrame({'contents': contents_data})
    return new_df

# 함수 호출
result_df_store = create_contents_rowwise(df_store)
print(result_df_store)

                       contents
0  '위치': 부산 부산진구 중앙대로 679-8 히스피
1           '전화번호': 01098007047
2         '결제수단': 현금, 신용카드, 포인트


### MENU_INFO

In [6]:
################ 전처리 ################
columns_to_use = ['STORE_NO', 'PROD_NM', 'CATEGORY_NM', 'PROD_BASE_PRICE']
df_menu = pd.read_excel(f'/home/user09/beaver/data/shared_files/dataset/dataset_v{CFG.version}.xlsx', sheet_name='MENU_INFO', usecols=columns_to_use)
df_menu = df_menu[df_menu['STORE_NO']==CFG.store_num]
df_menu.drop(columns=['STORE_NO'], inplace=True)

df_menu_col = df_menu.columns

print(df_menu[:3])
print(df_menu_col)

  CATEGORY_NM   PROD_NM  PROD_BASE_PRICE
0        시그니처  히스피 크림라떼             4900
1        시그니처  흑임자 크림라떼             4900
2        시그니처   곡물 크림라떼             4900
Index(['CATEGORY_NM', 'PROD_NM', 'PROD_BASE_PRICE'], dtype='object')


In [7]:
############## 컬럼명 변경 ##############
menu_column_mapping = {
    # 'PROD_NO': '상품 번호',
    'PROD_NM': '메뉴명',
    'CATEGORY_NM': '카테고리',
    'PROD_BASE_PRICE': '가격'
}

for col in df_menu_col:
    df_menu = df_menu.rename(columns={col: menu_column_mapping[col]})

df_menu['가격'] = df_menu['가격'].astype(str) + '원'
    
print(df_menu[:3])

   카테고리       메뉴명     가격
0  시그니처  히스피 크림라떼  4900원
1  시그니처  흑임자 크림라떼  4900원
2  시그니처   곡물 크림라떼  4900원


In [8]:
############### 한 컬럼으로 통합 ###############
def create_contents_rowwise(df):
    contents_data = []
    for _, row in df.iterrows():
        row_content = ", ".join([f"'{col}': {row[col]}" for col in df.columns])  # 각 행의 컬럼명:값을 ", "로 결합하여 하나의 문자열 생성
        contents_data.append(row_content)
    
    new_df = pd.DataFrame({'contents': contents_data})  # contents 열을 포함하는 새로운 데이터프레임 생성
    return new_df

# 함수 호출
result_df_menu = create_contents_rowwise(df_menu)
print(result_df_menu[:3])

                                     contents
0  '카테고리': 시그니처, '메뉴명': 히스피 크림라떼, '가격': 4900원
1  '카테고리': 시그니처, '메뉴명': 흑임자 크림라떼, '가격': 4900원
2   '카테고리': 시그니처, '메뉴명': 곡물 크림라떼, '가격': 4900원


### TIME_INFO

In [7]:
### 전처리 ###
columns_to_use = ['STORE_NO', 'WKDY', 'SALE_BGN_TM', 'SALE_END_TM', 'STORE_REST_BGN_TM', 'STORE_REST_END_TM']
df_time = pd.read_excel(f'/home/user09/beaver/data/shared_files/dataset/dataset_v{CFG.version}.xlsx', sheet_name='TIME_INFO', usecols=columns_to_use)
df_time = df_time[df_time['STORE_NO']==CFG.store_num]

# 1. 요일 매핑
day_mapping = {
    2: '월요일',
    3: '화요일',
    4: '수요일',
    5: '목요일',
    6: '금요일',
    7: '토요일',
    1: '일요일'
}
df_time['WKDY'] = df_time['WKDY'].map(day_mapping)
df_time.fillna(CFG.fill_nan, inplace=True)

def convert_time_format(hhmmss):
    if pd.isna(hhmmss) or hhmmss in [None, 'None']:
        return CFG.fill_nan
    hhmmss = str(int(hhmmss)).zfill(6)
    return hhmmss[:2] + ':' + hhmmss[2:4]

def handle_missing_values(df, columns):
    for col in columns:
        if df[col].isna().all():  # 해당 매장의 해당 컬럼의 모든 값이 없으면, 정보가 없는 것으로 간주하여 None값으로 채움
            df[col] = CFG.fill_nan 
        else:                     # 해당 매장의 해당 컬럼의 일부 값이 있으면, 정보가 없는 것이 아니라 해당 요일은 브레이크 타임이 없는 것으로 간주하여 "없음"으로 채움
            df[col] = df[col].fillna("없음")  # NaN인 부분만 "없음"으로 채움
    return df

# 시간 변환 함수 적용
df_time['SALE_BGN_TM'] = df_time['SALE_BGN_TM'].apply(convert_time_format)
df_time['SALE_END_TM'] = df_time['SALE_END_TM'].apply(convert_time_format)
df_time['STORE_REST_BGN_TM'] = df_time['STORE_REST_BGN_TM'].apply(convert_time_format)
df_time['STORE_REST_END_TM'] = df_time['STORE_REST_END_TM'].apply(convert_time_format)

# 빈값 처리 함수 적용
columns_to_check = ['SALE_BGN_TM', 'SALE_END_TM', 'STORE_REST_BGN_TM', 'STORE_REST_END_TM']
df_time = handle_missing_values(df_time, columns_to_check)

print(df_time)

   STORE_NO  WKDY SALE_BGN_TM SALE_END_TM STORE_REST_BGN_TM STORE_REST_END_TM
0    102496  None        None        None              None              None


In [8]:
# # 분리
# def process_business_hours(df):
#     grouped = df.groupby(
#         ["SALE_BGN_TM", "SALE_END_TM"]
#     )["WKDY"].apply(list).reset_index()
    
#     combined_data = {}
#     for _, row in grouped.iterrows():
#         if row["SALE_BGN_TM"] == CFG.fill_nan and row["SALE_END_TM"] == CFG.fill_nan:
#             continue
#         else:
#             # Flatten WKDY if nested lists are present
#             weekdays = "&".join([day for sublist in row["WKDY"] for day in sublist] if isinstance(row["WKDY"][0], list) else row["WKDY"])
#             # Add 영업시간 to combined_data
#             combined_data[f"'{weekdays}_영업시간'"] = f"{row['SALE_BGN_TM']}-{row['SALE_END_TM']}"

#     if not combined_data:
#         combined_data = CFG.fill_nan

#     # Format combined_data
#     formatted_combined_data = ", ".join([f"{key}: {value}" for key, value in combined_data.items()])

#     result = {"contents": formatted_combined_data}
#     new_df = pd.DataFrame([result])
#     return new_df

# # Apply the function
# business_hours_df = process_business_hours(df_time)
# print(business_hours_df)


In [9]:
def process_business_hours(df):
    grouped = df.groupby(
        ["SALE_BGN_TM", "SALE_END_TM"]
    )["WKDY"].apply(list).reset_index()
    
    combined_data = {}
    for _, row in grouped.iterrows():
        if row["SALE_BGN_TM"] == CFG.fill_nan and row["SALE_END_TM"] == CFG.fill_nan:
            continue
        else:
            # Flatten WKDY if nested lists are present
            weekdays = "&".join([day for sublist in row["WKDY"] for day in sublist] if isinstance(row["WKDY"][0], list) else row["WKDY"])
            # Add 영업시간 to combined_data
            combined_data[f"'{weekdays}_영업시간'"] = f"{row['SALE_BGN_TM']}-{row['SALE_END_TM']}"

    if not combined_data:
        combined_data = {}  # 빈 딕셔너리로 설정

    # Format combined_data
    formatted_combined_data = ", ".join([f"{key}: {value}" for key, value in combined_data.items()])

    if formatted_combined_data == "":
        formatted_combined_data = f"'영업시간':{CFG.fill_nan}"
    result = {"contents": formatted_combined_data}
    new_df = pd.DataFrame([result])
    return new_df

# Apply the function
business_hours_df = process_business_hours(df_time)
print(business_hours_df)


      contents
0  '영업시간':None


In [10]:
# # 분리
# def process_break_time(df):
#     grouped = df.groupby(
#         ["STORE_REST_BGN_TM", "STORE_REST_END_TM"]
#     )["WKDY"].apply(list).reset_index()
    
#     combined_data = {}
#     for _, row in grouped.iterrows():
#         if row["STORE_REST_BGN_TM"] == "없음" or row["STORE_REST_BGN_TM"] == CFG.fill_nan:
#             break_time = "브레이크타임없음"
#         else:
#             break_time = f"{row['STORE_REST_BGN_TM']}~{row['STORE_REST_END_TM']}"
        
#         # Flatten WKDY if nested lists are present
#         weekdays = "&".join([day for sublist in row["WKDY"] for day in sublist] if isinstance(row["WKDY"][0], list) else row["WKDY"])
#         # Add 브레이크타임 to combined_data
#         combined_data[f"'{weekdays}_브레이크타임'"] = break_time

#     if not combined_data:
#         combined_data = CFG.fill_nan

#     # Format combined_data
#     formatted_combined_data = ", ".join([f"{key}: {value}" for key, value in combined_data.items()])

#     result = {"contents": formatted_combined_data}
#     new_df = pd.DataFrame([result])
#     return new_df

# # Apply the function
# break_time_df = process_break_time(df_time)
# print(break_time_df)


In [13]:
def process_break_time_with_none(df):
    grouped = df.groupby(
        ["STORE_REST_BGN_TM", "STORE_REST_END_TM"]
    )["WKDY"].apply(list).reset_index()
    
    combined_data = {}
    for _, row in grouped.iterrows():
        if row["STORE_REST_BGN_TM"] == "없음" or row["STORE_REST_BGN_TM"] == CFG.fill_nan:
            break_time = "브레이크타임없음"
        else:
            break_time = f"{row['STORE_REST_BGN_TM']}~{row['STORE_REST_END_TM']}"

        # Handle None in WKDY
        if row["WKDY"] is None or all(day is None for day in row["WKDY"]):
            weekdays = "None"
        else:
            # Flatten WKDY if nested lists are present
            weekdays = "&".join([day for sublist in row["WKDY"] for day in sublist] if isinstance(row["WKDY"][0], list) else row["WKDY"])
        
        # Add 브레이크타임 to combined_data
        if weekdays == "None":
            combined_data["'브레이크타임'"] = None
        else:
            combined_data[f"'{weekdays}_브레이크타임'"] = break_time

    if not combined_data:
        combined_data = {"브레이크타임": "None"}

    # Format combined_data
    formatted_combined_data = ", ".join([f"{key}: {value}" for key, value in combined_data.items()])

    result = {"contents": formatted_combined_data}
    new_df = pd.DataFrame([result])
    return new_df

# Apply the function
break_time_df = process_break_time_with_none(df_time)
print(break_time_df)

         contents
0  '브레이크타임': None


In [14]:
# 합치기
result_df_time = pd.concat([business_hours_df, break_time_df], ignore_index=True)
print(result_df_time)

         contents
0     '영업시간':None
1  '브레이크타임': None


In [15]:
# # 전체 합친거
# def group_by_schedule_to_single_row_fixed(df):
#     grouped = df.groupby(
#         ["SALE_BGN_TM", "SALE_END_TM", "STORE_REST_BGN_TM", "STORE_REST_END_TM"]
#     )["WKDY"].apply(list).reset_index()
    
#     combined_data = {}
#     for _, row in grouped.iterrows():
#         if (
#             row["SALE_BGN_TM"] == CFG.fill_nan and 
#             row["SALE_END_TM"] == CFG.fill_nan and 
#             row["STORE_REST_BGN_TM"] == CFG.fill_nan and 
#             row["STORE_REST_END_TM"] == CFG.fill_nan
#         ):
#             continue
#         else:
#             if row["STORE_REST_BGN_TM"] == "없음" or row["STORE_REST_BGN_TM"] == CFG.fill_nan:
#                 break_time = "브레이크타임없음"
#             else:
#                 break_time = f"{row['STORE_REST_BGN_TM']}~{row['STORE_REST_END_TM']}"
            
#             # Flatten WKDY if nested lists are present
#             weekdays = "&".join([day for sublist in row["WKDY"] for day in sublist] if isinstance(row["WKDY"][0], list) else row["WKDY"])
            
#             # Add formatted data to combined_data
#             combined_data[f"'{weekdays}_영업시간'"] = f"{row['SALE_BGN_TM']}-{row['SALE_END_TM']}"
#             combined_data[f"'{weekdays}_브레이크타임'"] = break_time

#     if not combined_data:
#         combined_data = CFG.fill_nan

#     formatted_combined_data = (
#         "{" +
#         ", ".join([f"{key}: {value}" for key, value in combined_data.items()]) +
#         "}"
#     )

#     result = {"contents": formatted_combined_data}
#     new_df = pd.DataFrame([result])
#     return new_df

# # Apply the function
# result_df_time = group_by_schedule_to_single_row_fixed(df_time)

# print(result_df_time)


### 휴무일

In [16]:
columns_to_use = ['STORE_NO', 'HOLIDAY_TYPE_CD']
df_holiday = pd.read_excel(f'/home/user09/beaver/data/shared_files/dataset/dataset_v{CFG.version}.xlsx', sheet_name='HOLIDAY_INFO', usecols=columns_to_use)
df_holiday = df_holiday[df_holiday['STORE_NO']==CFG.store_num]   # 해당 매장 정보만 가져옴
df_holiday.fillna(CFG.fill_nan, inplace=True)
df_holiday.drop(columns=['STORE_NO'], inplace=True)
df_holiday_col = df_holiday.columns
print(df_holiday)

  HOLIDAY_TYPE_CD
0            None


In [17]:
############## 컬럼명 변경 ##############
day_column_mapping = {
    # 'PROD_NO': '상품 번호',
    'HOLIDAY_TYPE_CD': '휴무일',
}

for col in df_holiday_col:
    df_holiday = df_holiday.rename(columns={col: day_column_mapping[col]})
    
print(df_holiday[:3])

    휴무일
0  None


In [18]:
############### 한 컬럼으로 통합 ###############
def create_contents_rowwise(df):
    contents_data = []
    for _, row in df.iterrows():
        row_content = ", ".join([f"'{col}': {row[col]}" for col in df.columns])  # 각 행의 컬럼명:값을 ", "로 결합하여 하나의 문자열 생성
        contents_data.append(row_content)
    
    new_df = pd.DataFrame({'contents': contents_data})  # contents 열을 포함하는 새로운 데이터프레임 생성
    return new_df

# 함수 호출
result_df_holiday = create_contents_rowwise(df_holiday)
print(result_df_holiday[:3])

      contents
0  '휴무일': None


### 4개 통합 & 피클파일

In [19]:
df = pd.concat([result_df_time, result_df_store, result_df_menu, result_df_holiday], ignore_index=True)

In [20]:
print(df)

                                         contents
0                                     '영업시간':None
1                                  '브레이크타임': None
2                    '위치': 부산 부산진구 중앙대로 679-8 히스피
3                             '전화번호': 01098007047
4                           '결제수단': 현금, 신용카드, 포인트
..                                            ...
256           '카테고리': 시즌, '메뉴명': 옥스피, '가격': 4400원
257        '카테고리': 시즌, '메뉴명': 달밤크림라떼, '가격': 5500원
258       '카테고리': 시즌, '메뉴명': 달밤치즈프라페, '가격': 5900원
259  '카테고리': 시즌, '메뉴명': 가을 디저트 선물세트, '가격': 16900원
260                                   '휴무일': None

[261 rows x 1 columns]


### 기본 정보 지정

In [21]:
# ":"를 기준으로 분리하여 앞의 키워드를 추출한 후, 존재 여부 확인
# existing_info = [content.split(":")[0].strip() for content in df["contents"]]
# missing_info = [info for info in CFG.basic_info if info not in existing_info]
# for info in missing_info:
#     df = pd.concat([df, pd.DataFrame({"contents": [f"{info}: 정보없음"]})], ignore_index=True)

In [22]:
# 결과 저장 및 출력
df.to_excel(CFG.save_path, index=False)   # 전처리한 엑셀 파일 저장
print(df)

                                         contents
0                                     '영업시간':None
1                                  '브레이크타임': None
2                    '위치': 부산 부산진구 중앙대로 679-8 히스피
3                             '전화번호': 01098007047
4                           '결제수단': 현금, 신용카드, 포인트
..                                            ...
256           '카테고리': 시즌, '메뉴명': 옥스피, '가격': 4400원
257        '카테고리': 시즌, '메뉴명': 달밤크림라떼, '가격': 5500원
258       '카테고리': 시즌, '메뉴명': 달밤치즈프라페, '가격': 5900원
259  '카테고리': 시즌, '메뉴명': 가을 디저트 선물세트, '가격': 16900원
260                                   '휴무일': None

[261 rows x 1 columns]


In [23]:
#### 엑셀파일 DB화(pickle파일로 변환) ##### Extracting information from the "Details" column to create documents
docs = []
for _, row in df.iterrows():
    details = row['contents']
    # metadata = {}
    
    # # Parse key-value pairs in the "Details" column
    # for detail in details.split('\n'):
    #     if ':' in detail:
    #         key, value = detail.split(':', 1)
            # metadata[key.strip()] = value.strip()
    
    docs.append(Document(page_content=details))#, metadata=metadata))

# Embeddings and vector store setup
encode_kwargs = {'normalize_embeddings': True}
model_kwargs = {'device': 'cpu'}

hf = HuggingFaceEmbeddings(
    model_name=CFG.embedding_model,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)

db = FAISS.from_documents(docs, hf)

# Save the FAISS vector store and documents as pickle
db.save_local(f"{CFG.output_path}/{CFG.store_num}_faiss{CFG.version}")

with open(f"{CFG.output_path}/{CFG.store_num}_docs{CFG.version}.pkl", "wb") as f:
    pickle.dump(docs, f)

# Load the FAISS vector store
db = FAISS.load_local(f"/home/user09/beaver/data/db/{CFG.store_num}_faiss{CFG.version}", hf, allow_dangerous_deserialization=True)

# Create retrievers
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": CFG.retriever_k})
bm25_retriever = BM25Retriever.from_documents(docs)
bm25_retriever.k = CFG.retriever_k

ensemble_retriever = EnsembleRetriever(
    retrievers=[retriever, bm25_retriever],
    weights=[CFG.retriever_bert_weight, 1 - CFG.retriever_bert_weight]
)


/home/user09/venv/beaver/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
